# SK Summarizer Pattern

Learning objectives:

- Understand a simple Summarization Implementation

## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.3.0"
#r "nuget: dotenv.net"

using System;
using System.IO;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using dotenv.net;

#!import Models/Models.cs
#!import Utils/TextChunker.cs
#!import Templates/Templates.cs

const int ChunkSize = 1000;
const int MaxTokens = 1500;
const double Temperature = 0.3;

### Read the API Key and endpoints from environment variables or the .env file

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var adaDeploymentName = "ada";
var checkKeyAndEndpoint = $"Endpoint: {endpoint}{deploymentName} with key {apiKey.Substring(0, 5)}...";
checkKeyAndEndpoint

### Get a Semantic Kernel instance configured for Chat completions

In [ ]:
var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey)
    .Build();

## Ingestion

### Read transcript from a file

In [ ]:
var text = File.ReadAllText("data/transcript.txt");
text

## Chunk the text

In [ ]:
var chunks = TextChunker.ChunkText(text, ChunkSize);
chunks

## Process the chunk summaries

In [ ]:
foreach(var chunk in chunks)
{
    // REMINDER: Templates are in the Templates/Templates.cs file
    var prompt = SummaryTemplate.Replace("{context}", chunk.Content);
    // Create a fixed SK function
    var fixedFunction = kernel.CreateFunctionFromPrompt(prompt);
    // Run the function
    var result = await kernel.InvokeAsync(fixedFunction);
    chunk.Summary = result.ToString();
}
chunks

## Final summary

In [ ]:
var augmentedContext = new StringBuilder();

foreach(var chunk in chunks)
{
    augmentedContext.Append(chunk.Summary+"\n\n");    
}

// REMINDER: Templates are in the Templates/Templates.cs file
var funcTemplate = FinalSummaryTemplate.Replace("{context}", augmentedContext.ToString());

// Create a Prompt SK function from the template
var fixedFunction = kernel.CreateFunctionFromPrompt(funcTemplate,new OpenAIPromptExecutionSettings() { MaxTokens = MaxTokens, Temperature = Temperature, TopP = 1 });

// Run the function
var result = await kernel.InvokeAsync(fixedFunction);
result.ToString()